# WOfS Validation Data Analysis_Point-based  <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[ga_ls8c_wofs_2](https://explorer.digitalearth.africa/ga_ls8c_wofs_2),
[ga_ls8c_wofs_2_summary ](https://explorer.digitalearth.africa/ga_ls8c_wofs_2_summary)

## Background
The [Water Observations from Space (WOfS)](https://www.ga.gov.au/scientific-topics/community-safety/flood/wofs/about-wofs) is a derived product from Landsat 8 satellite observations as part of provisional Landsat 8 Collection 2 surface reflectance and shows surface water detected in Africa.
Individual water classified images are called Water Observation Feature Layers (WOFLs), and are created in a 1-to-1 relationship with the input satellite data. 
Hence there is one WOFL for each satellite dataset processed for the occurrence of water.

The data in a WOFL is stored as a bit field. This is a binary number, where each digit of the number is independantly set or not based on the presence (1) or absence (0) of a particular attribute (water, cloud, cloud shadow etc). In this way, the single decimal value associated to each pixel can provide information on a variety of features of that pixel. 
For more information on the structure of WOFLs and how to interact with them, see [Water Observations from Space](../Datasets/Water_Observations_from_Space.ipynb) and [Applying WOfS bitmasking](../Frequently_used_code/Applying_WOfS_bitmasking.ipynb) notebooks. 

## Description
This notebook explains how you can perform accuracy assessment for WOFS derived product using collected ground truth dataset. 

The notebook demonstrates how to:

1. Load validation points for each partner institutions following cleaning stage as an ESRI shapefile
2. Query WOFL data for validation points and capture available WOfS observation available
3. Extract statistics for each WOfS observation in each validation point 
4. Extract a LUT for each point that contains both validation info and WOfS result for each month 
***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

After finishing the analysis, you can modify some values in the "Analysis parameters" cell and re-run the analysis to load WOFLs for a different location or time period.

### Load packages
Import Python packages that are used for the analysis.

In [21]:
%matplotlib inline

import time 
import datacube
from datacube.utils import masking, geometry 
import sys
import os
import dask 
import rasterio, rasterio.features
import xarray
import glob
import numpy as np
import pandas as pd
import seaborn as sn
import geopandas as gpd
import subprocess as sp
import matplotlib.pyplot as plt
import scipy, scipy.ndimage
import warnings
warnings.filterwarnings("ignore") #this will suppress the warnings for multiple UTM zones in your AOI 

sys.path.append("../Scripts")
from rasterio.mask import mask
from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point
from deafrica_plotting import map_shapefile,display_map, rgb
from deafrica_spatialtools import xr_rasterize
from deafrica_datahandling import wofs_fuser, mostcommon_crs,load_ard,deepcopy
from deafrica_dask import create_local_dask_cluster

#for parallelisation 
from multiprocessing import Pool, Manager
import multiprocessing as mp
from tqdm import tqdm

### Connect to the datacube
Activate the datacube database, which provides functionality for loading and displaying stored Earth observation data.

In [22]:
dc = datacube.Datacube(app='WOfS_accuracy')

### Analysis parameters

To analyse validation points collected by each partner institution, we need to obtain WOfS surface water observation data that corresponds with the labelled input data locations.

### Loading Dataset

1. Load validation points for each partner institutions as a list of observations each has a location and month
    * Load the cleaned validation file as ESRI `shapefile`
    * Inspect the shapefile

In [25]:
path = '../Supplementary_data/Validation/Refined/groundtruth_AFRIGIST.shp'

In [26]:
#open shapefile and ensure its in WGS84 coordinates
input_data = gpd.read_file(path).to_crs('epsg:4326')
#check the shapfile by plotting it
#map_shapefile(input_data, attribute='CLASS')

In [27]:
input_data.columns

Index(['Unnamed_ 0', 'Unnamed__1', 'PLOT_ID', 'LON', 'LAT', 'FLAGGED',
       'ANALYSES', 'SENTINEL2Y', 'STARTDATE', 'ENDDATE', 'WATER', 'NO_WATER',
       'BAD_IMAGE', 'NOT_SURE', 'CLASS', 'COMMENT', 'MONTH', 'WATERFLAG',
       'geometry'],
      dtype='object')

In [29]:
#clean the first two columns
input_data= input_data.drop(['Unnamed_ 0','Unnamed__1'], axis=1)

In [35]:
input_data

,PLOT_ID,LON,LAT,FLAGGED,ANALYSES,SENTINEL2Y,STARTDATE,ENDDATE,WATER,NO_WATER,...,NOT_SURE,CLASS,COMMENT,MONTH,WATERFLAG,geometry,CLASS_WET,CLASS_DRY,CLEAR_OBS,FREQUENCY
0,137483175.0,30.463813,-26.653807,0.0,1.0,2018,None,None,1-12,0,...,0,Open water - freshwater,None,1,1,POINT (30.46381 -26.65381),0.0,0.0,0.0,NaN
1,137483175.0,30.463813,-26.653807,0.0,1.0,2018,None,None,1-12,0,...,0,Open water - freshwater,None,2,1,POINT (30.46381 -26.65381),0.0,0.0,0.0,NaN
2,137483175.0,30.463813,-26.653807,0.0,1.0,2018,None,None,1-12,0,...,0,Open water - freshwater,None,3,1,POINT (30.46381 -26.65381),0.0,0.0,0.0,0.0
3,137483175.0,30.463813,-26.653807,0.0,1.0,2018,None,None,1-12,0,...,0,Open water - freshwater,None,4,1,POINT (30.46381 -26.65381),0.0,0.0,0.0,0.0
4,137483175.0,30.463813,-26.653807,0.0,1.0,2018,None,None,1-12,0,...,0,Open water - freshwater,None,5,1,POINT (30.46381 -26.65381),0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11329,137482804.0,13.838962,-9.369115,0.0,1.0,2018,1/09/2018,5/09/2018,8,"1-7,9-12",...,0,Open water - freshwater,na,7,2,POINT (13.83896 -9.36912),NaN,NaN,NaN,NaN
11330,137482804.0,13.838962,-9.369115,0.0,1.0,2018,1/09/2018,5/09/2018,8,"1-7,9-12",...,0,Open water - freshwater,na,9,2,POINT (13.83896 -9.36912),NaN,NaN,NaN,NaN
11331,137482804.0,13.838962,-9.369115,0.0,1.0,2018,1/09/2018,5/09/2018,8,"1-7,9-12",...,0,Open water - freshwater,na,10,2,POINT (13.83896 -9.36912),NaN,NaN,NaN,NaN
11332,137482804.0,13.838962,-9.369115,0.0,1.0,2018,1/09/2018,5/09/2018,8,"1-7,9-12",...,0,Open water - freshwater,na,11,2,POINT (13.83896 -9.36912),NaN,NaN,NaN,NaN


In [30]:
coords = [(x,y) for x, y in zip(input_data.geometry.x, input_data.geometry.y)]

In [31]:
pd.date_range('01-2018','01-2019', freq='M')

DatetimeIndex(['2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30',
               '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31',
               '2018-09-30', '2018-10-31', '2018-11-30', '2018-12-31'],
              dtype='datetime64[ns]', freq='M')

### Sample WOfS at the ground truth coordinates 
To load WOFL data, we can first create a re-usable query as below that will define the time period we are interested in, as well as other important parameters that are used to correctly load the data. 

As WOFLs are created scene-by-scene, and some scenes overlap, it's important when loading data to `group_by` solar day, and ensure that the data between scenes is combined correctly by using the WOfS `fuse_func`.
This will merge observations taken on the same day, and ensure that important data isn't lost when overlapping datasets are combined.

We can convert the WOFL bit field into a binary array containing True and False values. This allows us to use the WOFL data as a mask that can be applied to other datasets.
The `make_mask` function allows us to create a mask using the flag labels (e.g. "wet" or "dry") rather than the binary numbers we used above. For more details on how to do masking on WOfS, see the [Applying_WOfS_bit_masking](../Frequently_used_code/Applying_WOfS_bitmasking.ipynb) notebook.

In [32]:
#generate query object 
query ={'resolution':(-30, 30),
        'align':(15,15),
        'group_by':'solar_day'}

In [33]:
# Define a mask for wet and clear pixels 
wet_nocloud = {"water_observed":True, "cloud_shadow":False, "cloud":False,"nodata":False}
# Define a mask for dry and clear pixels 
dry_nocloud = {"water_observed":False, "cloud_shadow":False, "cloud":False, "nodata":False}

In [1]:
##Step 1: update the query for WOfS based on CEO input table 

for index, row in input_data.iterrows():
    #get the month value for each index
    month = input_data.loc[index]['MONTH'] 
    #set the time for query of the WOfS database according to the month value in the validation table 
    #time = '2018-' + f'{month:02d}'
    start_month = '2018-'+f'{month:02d}'+'-01'
    end_month = '2018-'+f'{month:02d}'+'-05'
    time = (start_month,end_month)
    t = {"time":time} 
    #print(t)
    #this is for having the original query as it is 
    dc_query = deepcopy(query) 
    geom = geometry.Geometry(input_data.geometry.values[index].__geo_interface__, geometry.CRS('epsg:4326')) 
    q = {"geopolygon":geom}
    dc_query.update(t)
    dc_query.update(q)
    wofls = dc.load(product ="ga_ls8c_wofs_2", output_crs = 'EPSG:6933', fuse_func=wofs_fuser,**dc_query).squeeze()
    if not 'water' in wofls:
         #update the column for the clear and water observation 
        input_data.at[index,'CLASS_WET'] = 0
    #update the column for the clear and no water observaion 
        input_data.at[index,'CLASS_DRY'] = 0
    #update the column for the clear water observation frequency 
 
    #update the column for the clear observation 
        input_data.at[index,'CLEAR_OBS'] = 0
    #update the column for the frequency of clear and wet observations 
        input_data.at[index,'FREQUENCY'] = 0
##Step 2: update the table with count of clear observations that are either wet or dry in that particular month 
    else:
    #mask the wofls for wet and clear pixels 
        wofl_wetnocloud = masking.make_mask(wofls, **wet_nocloud).astype(int) 
        wofl_wet = wofl_wetnocloud.water.sum() 
        #mask the wofls for dry and clean pixels 
        wofl_drynocloud = masking.make_mask(wofls, **dry_nocloud).astype(int)
        wofl_dry = wofl_drynocloud.water.sum()

    #update the column for the clear and water observation 
        input_data.at[index,'CLASS_WET'] = wofl_wet.values 
        #update the column for the clear and no water observaion 
        input_data.at[index,'CLASS_DRY'] = wofl_dry.values
        #update the column for the clear water observation frequency 
        wofl_clear_observations = wofl_dry + wofl_wet 
        #update the column for the clear observation 
        input_data.at[index,'CLEAR_OBS'] = (wofl_clear_observations).values
        #update the column for the frequency of clear and wet observations 
        input_data.at[index,'FREQUENCY'] = (wofl_wet / wofl_clear_observations).values 

In [3]:
#for checking the outcome of the second filtering with a need to run the entire processing (in above cell)
CEO = '../Supplementary_data/Validation/Refined/ground_truth_RCMRD.csv'
input_data= pd.read_csv(CEO,delimiter=",")

In [11]:
indexNames = (input_data['WATERFLAG'] > 2) & (input_data['CLEAR_OBS'] > 0) & (input_data['FREQUENCY'] > 0) & (input_data['FREQUENCY'] < 1) 
indexNames.sum()

13

In [14]:
#as water flag more than 1 and also clear observation equal to zero 
indexNames = input_data[(input_data['WATERFLAG'] > 1) | (input_data['CLEAR_OBS'] == 0)].index
input_data.drop(indexNames, inplace=True)

In [1]:
#input_data

In [22]:
#this will count on the number of month for each plotID in the final table 
count = input_data.groupby(['PLOT_ID'])['MONTH'].count()

In [23]:
#save the check on the count for each plot id as a csv file 
count.to_csv('../Supplementary_data/Validation/Refined/final_RCMRD_count.csv')

In [24]:
input_data.to_csv(('../Supplementary_data/Validation/Refined/ground_truth_RCMRD_final.csv'))

In [2]:
print(datacube.__version__)

1.8.2.dev7+gdcab0e02


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** January 2020

**Compatible datacube version:** 

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)